In [ ]:
!pip install llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.11.2
    Uninstalling beautifulsoup4-4.11.2:
      Successfully uninst

In [ ]:
import os
from IPython.display import Markdown, display
import pandas as pd
import numpy as np
import sqlite3

#Creating Schema and DB

In [ ]:
# Excel fiel path

data = "data.xlsx"

--2023-12-27 06:37:57--  https://final-output-llm.s3.us-west-2.amazonaws.com/data.xlsx
Resolving final-output-llm.s3.us-west-2.amazonaws.com (final-output-llm.s3.us-west-2.amazonaws.com)... 3.5.83.0, 52.92.192.170, 52.92.209.18, ...
Connecting to final-output-llm.s3.us-west-2.amazonaws.com (final-output-llm.s3.us-west-2.amazonaws.com)|3.5.83.0|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44172 (43K) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘data.xlsx’

data.xlsx           100%[===================>]  43.14K  --.-KB/s    in 0.1s    

2023-12-27 06:37:57 (316 KB/s) - ‘data.xlsx’ saved [44172/44172]



In [ ]:
from sqlalchemy import insert
import pandas as pd
from llama_index import SQLDatabase, ServiceContext
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    Float,
    ForeignKey,
)
import sqlalchemy


#  create Engine
engine = create_engine("sqlite:///database.sqlite")
metadata_obj = MetaData()


# create SQL Tables
car_data_var = Table(
    "cars_data",
    metadata_obj,
    Column("Id", Integer, primary_key=True),
    Column("MPG", Integer),
    Column("Cylinders", Integer),
    Column("Edispl", Integer),
    Column("Horsepower",Integer),
    Column("Weight", Integer),
    Column("Accelerate", Float),
    Column("Year", Integer),
)

car_names_var = Table(
    "car_names",
    metadata_obj,
    Column("Id", Integer, primary_key=True),
    Column("Model", String(30)),
    Column("Make", String(30)),
)


car_continents_var = Table(
    "car_continents",
    metadata_obj,
    Column("ContId", Integer, primary_key=True),
    Column("Continent", String(30)),
)

car_countries_var = Table(
    "car_countries",
    metadata_obj,
    Column("CountryId", Integer, primary_key=True),
    Column("CountryName", String(30)),
    Column("Continent", Integer,ForeignKey("car_continents.ContId"), nullable=False),
)


car_makers_var = Table(
    "car_makers",
    metadata_obj,
    Column("Id", Integer, primary_key=True),
    Column("Maker", String(16)),
    Column("FullName", String(30)),
    Column("Country", Integer,ForeignKey("car_countries.CountryId")),
    Column("Continent",Integer,ForeignKey("car_continents.ContId"))
)



languages_data_var = Table(
    "languages_data",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(16))
)

languages_countries_var = Table(
    "languages_countries",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(16)),
    Column("overall_score", Float),
    Column("justice_score", Float),
)

language_official_languages_var = Table(
    "language_official_languages",
    metadata_obj,
    Column("language_id", Integer, ForeignKey("languages_data.id"), nullable=False),
    Column("country_id", Integer, ForeignKey("languages_countries.id"), nullable=False),
)



metadata_obj.create_all(engine)




sql_database = SQLDatabase(engine, include_tables=["cars_data","car_makers","car_names","car_continents","car_countries","languages_data","languages_countries","language_official_languages"])


# Read excel fiel and create dictionary from them
xls = pd.ExcelFile("/content/data.xlsx")
cars_data_df = pd.read_excel(xls, 'cars-data')
car_makers_df = pd.read_excel(xls, 'car-makers')
cars_names_df = pd.read_excel(xls, 'car-names')
cars_continents_df = pd.read_excel(xls, 'car-continents')
car_countries_df = pd.read_excel(xls, 'car-countries')
languages_data_df = pd.read_excel(xls, 'languages-data')
languages_countries_df = pd.read_excel(xls, 'languages-countries')
languages_official_languages_df = pd.read_excel(xls, 'languages-official_languages')



rows_cars_data = cars_data_df.drop("Id",axis=1).to_dict(orient='records')
rows_car_makers = car_makers_df.drop("Id",axis=1).to_dict(orient='records')
rows_cars_names = cars_names_df.drop("Id",axis=1).to_dict(orient='records')
rows_cars_continents = cars_continents_df.to_dict(orient='records')
rows_car_countries = car_countries_df.to_dict(orient='records')
rows_languages_data = languages_data_df.to_dict(orient='records')
rows_languages_countries = languages_countries_df.to_dict(orient='records')
rows_languages_official_languages = languages_official_languages_df.to_dict(orient='records')


# function for inserting data into table
def insert_data(row_name,table_name):
  for row in row_name:
    stmt = insert(table_name).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)


# insert data into table
insert_data(rows_cars_data,car_data_var)
insert_data(rows_car_makers,car_makers_var)
insert_data(rows_cars_names,car_names_var)
insert_data(rows_cars_continents,car_continents_var)
insert_data(rows_car_countries,car_countries_var)
insert_data(rows_languages_data,languages_data_var)
insert_data(rows_languages_countries,languages_countries_var)
insert_data(rows_languages_official_languages,language_official_languages_var)

In [ ]:
# check if data is insert or not

stmt = select(
    car_makers_var.c.Maker,
    car_makers_var.c.FullName,
    car_makers_var.c.Country,
).select_from(car_data_var)

with engine.connect() as connection:
    results = connection.execute(stmt).fetchall()
    print(results)


print(sql_database._all_tables)
list(sql_database._all_tables)

<ipython-input-5-0a7e5080030e>:10: SAWarning: SELECT statement has a cartesian product between FROM element(s) "car_makers" and FROM element "cars_data".  Apply join condition(s) between each element to resolve.
  results = connection.execute(stmt).fetchall()


[("'amc'", "'American Motor Company'", 1), ("'volkswagen'", "'Volkswagen'", 2), ("'bmw'", "'BMW'", 2), ("'gm'", "'General Motors'", 1), ("'ford'", "'Ford Motor Company'", 1), ("'chrysler'", "'Chrysler'", 1), ("'citroen'", "'Citroen'", 3), ("'nissan'", "'Nissan Motors'", 4), ("'fiat'", "'Fiat'", 5), ("'hi'", "'hi'", None), ("'honda'", "'Honda'", 4), ("'mazda'", "'Mazda'", 4), ("'daimler benz'", "'Daimler Benz'", 2), ("'opel'", "'Opel'", 2), ("'peugeaut'", "'Peugeaut'", 3), ("'renault'", "'Renault'", 3), ("'saab'", "'Saab'", 6), ("'subaru'", "'Subaru'", 4), ("'toyota'", "'Toyota'", 4), ("'triumph'", "'Triumph'", 7), ("'volvo'", "'Volvo'", 6), ("'kia'", "'Kia Motors'", 8), ("'hyundai'", "'Hyundai'", 8), ("'amc'", "'American Motor Company'", 1), ("'volkswagen'", "'Volkswagen'", 2), ("'bmw'", "'BMW'", 2), ("'gm'", "'General Motors'", 1), ("'ford'", "'Ford Motor Company'", 1), ("'chrysler'", "'Chrysler'", 1), ("'citroen'", "'Citroen'", 3), ("'nissan'", "'Nissan Motors'", 4), ("'fiat'", "'Fia

['car_makers',
 'language_official_languages',
 'languages_countries',
 'languages_data',
 'cars_data',
 'car_names',
 'car_continents',
 'car_countries']

#Langchain SQL Agent

In [ ]:
!pip install langchain-experimental

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.0/163.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.4/794.4 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/192.4 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 6.2 MB/s eta 0:00:00


In [ ]:
!pip install -q python-dotenv chainlit
from langchain import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
import sqlite3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.6/105.6 kB 8.5 MB/s eta 0:00:

In [ ]:

import os
from dotenv import load_dotenv
import chainlit as cl
from langchain import PromptTemplate, SQLDatabase, HuggingFaceHub
from langchain_experimental.sql import SQLDatabaseChain

hf_api_token = 'Your huggingface API'

# repo = 'codellama/CodeLlama-7b-Instruct-hf'
repo = 'mistralai/Mistral-7B-Instruct-v0.2'

template = """You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
            Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
            Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
            Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
            Pay attention to use date('now') function to get the current date, if the question involves "today".

            Use the following format:

            Question: Question here
            SQLQuery: SQL Query to run
            SQLResult: Result of the SQLQuery
            Answer: Final answer here

            Only use the following tables:
            {table_info}

            Question: {input}"""


prompt = PromptTemplate(template=template, input_variables=["input","table_info","top_k"])

db = SQLDatabase.from_uri('sqlite:///database.sqlite')

llm = HuggingFaceHub(repo_id=repo, huggingfacehub_api_token=hf_api_token, model_kwargs={"temperature": 0.1, "max_length": 4096})


# sql_chain = SQLDatabaseChain.from_llm(prompt = prompt,llm=llm, db=db, verbose=True)  # with using Prompt Template


sql_chain = SQLDatabaseChain.from_llm(llm=llm, db=db, verbose=True)  # without using Prompt Template


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [ ]:
sql_chain.run('What is the number of cars with more than 4 cylinders?')



> Entering new SQLDatabaseChain chain...
What is the number of cars with more than 4 cylinders?
SQLQuery:SELECT COUNT(*) FROM cars_data WHERE Cylinders > 4;
SQLResult: [(195,)]
Answer:There are 195 cars with more than 4 cylinders.
> Finished chain.


'There are 195 cars with more than 4 cylinders.'

In [ ]:
sql_chain.run('For each continent, how many countries are there?')



> Entering new SQLDatabaseChain chain...
For each continent, how many countries are there?
SQLQuery:SELECT car_continents."Continent", COUNT(*) as "NumberOfCountries" FROM car_continents 
INNER JOIN car_countries ON car_continents."ContId" = car_countries."Continent" 
GROUP BY car_continents."Continent"
SQLResult: [("'africa'", 2), ("'america'", 3), ("'asia'", 2), ("'australia'", 2), ("'europe'", 6)]
Answer:There are 2 countries in Africa, 3 in America, 2 in Asia, and 2 in Australia. Europe has 6 countries.
> Finished chain.


'There are 2 countries in Africa, 3 in America, 2 in Asia, and 2 in Australia. Europe has 6 countries.'

In [ ]:
sql_chain.run('Which countries in europes continents have atleast 3 car manufacture?')



> Entering new SQLDatabaseChain chain...
Which countries in europes continents have atleast 3 car manufacture?
SQLQuery:SELECT c.CountryName FROM car_countries c JOIN car_makers m ON c.CountryId = m.Country JOIN car_names n ON m.Id = n.Make GROUP BY c.CountryName HAVING COUNT(DISTINCT m.Id) >= 3 AND c.Continent = 'europe';
SQLResult: 
Answer:The countries in Europe with at least 3 car manufacturers are:
1. Germany
2. France
SQLQuery:SELECT c.CountryName FROM car_countries c JOIN car_makers m ON c.CountryId = m.Country JOIN car_names n ON m.Id = n.Make GROUP BY c.CountryName HAVING COUNT(DISTINCT m.Id) >= 3 AND c.Continent = 'europe'
> Finished chain.


"The countries in Europe with at least 3 car manufacturers are:\n1. Germany\n2. France\nSQLQuery:SELECT c.CountryName FROM car_countries c JOIN car_makers m ON c.CountryId = m.Country JOIN car_names n ON m.Id = n.Make GROUP BY c.CountryName HAVING COUNT(DISTINCT m.Id) >= 3 AND c.Continent = 'europe'"

In [ ]:
sql_chain.run('What is the average justice score in the countries where English is not the official language?')



> Entering new SQLDatabaseChain chain...
What is the average justice score in the countries where English is not the official language?
SQLQuery:SELECT AVG(l.justice_score) FROM languages_countries l WHERE l.name != 'English';
SQLResult: [(87.54444444444444,)]
Answer:The average justice score in the countries where English is not the official language is approximately 87.54.
> Finished chain.


'The average justice score in the countries where English is not the official language is approximately 87.54.'